In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions_labor_distribution

# Step 1: Get the file path

In [2]:
report_path = helper_functions_labor_distribution.get_report_file_path('EmployeeAllocation')

report_path 

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='EmployeeAllocation']"

## Step 2: Retrieve the report and export as a CSV

In [3]:
df = helper_functions_labor_distribution.retrieve_employee_allocation_report(report_path) 
df 

,Employee,Employee Name,Job,Pay Group,Location,Location Code,Annual,Hourly,Employment Status,Allocation,Job Code,Location Code2,Location2,Org Level 3 Code,Scheduled Work Hours
0,001007204,Aasal_ Ashley E.,Rev Cycle Pymt Poster,Hourly BiWeekly,CSO - Payment Posting,000181,56284.8,27.06,Active,1,130142,000181,CSO - Payment Posting,000,80
1,001008549,Abafo_ Johanna A.,ASC Manager,Salaried Biweekly,PRS - ASC,002601,114400,55,Active,1,240001,002601,PRS - ASC,000,80
2,001005805,Abbott_ Deanna L.,Revenue Cycle Specialist,Hourly BiWeekly,EDOC - Ankle & Foot,002540,31200,25,Active,1,110161,002540,EDOC - Ankle & Foot,000,48
3,001006511,Abuachi_ Chimaroke I.,RN,Hourly BiWeekly,CSO - Valley ASC,000112,87318.4,41.98,Active,1,120001,000112,CSO - Valley ASC,000,80
4,001000432,Acosta_ Grace Lynne M.,Revenue Cycle Specialist,Hourly BiWeekly,BENT - Issaquah,007944,0,29.85,Active,0,110161,007944,BENT - Issaquah,000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,001006953,Zanavich_ David D.,Radiology Technologist,Hourly BiWeekly,PHSC,001100,0,35,Active,1,110311,001100,PHSC,000,0
2554,001008545,Zhang_ Jasmine L.,RN,Hourly BiWeekly,MJSS - ASC,004000,0,50,Active,1,120001,004000,MJSS - ASC,000,0
2555,001007823,Zornes_ Cosette M.,LPN,Hourly BiWeekly,ECOS,001000,57954,37.15,Active,1,120011,001000,ECOS,000,60
2556,001001109,Zundel_ Roger S.,Physician,Doctor BiWeekly,BENT,007900,39648,19.061538,Active,1,300,007900,BENT,3016,80


In [4]:
df.columns

Index(['Employee', 'Employee Name', 'Job', 'Pay Group', 'Location',
       'Location Code', 'Annual', 'Hourly', 'Employment Status', 'Allocation',
       'Job Code', 'Location Code2', 'Location2', 'Org Level 3 Code',
       'Scheduled Work Hours'],
      dtype='object')

In [5]:
#Remove the null values so that they say 0 instead
df['Location Code'] = df['Location Code'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Annual'] = df['Annual'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Hourly'] = df['Hourly'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Allocation'] = df['Allocation'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Job Code'] = df['Job Code'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Location Code2'] = df['Location Code2'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Org Level 3 Code'] = df['Org Level 3 Code'].astype(str).replace("'@xs:nil': 'true'", '0')
df['Scheduled Work Hours'] = df['Scheduled Work Hours'].astype(str).replace("'@xs:nil': 'true'", '0')

# Step 3: Export the report as a CSV

In [6]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [7]:
helper_functions_labor_distribution.create_sql_table_v2('ZZ_TEST_EmployeeAllocation')

# Step 5: Close the API connection

In [8]:
helper_functions_labor_distribution.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'

The code to create the SQL Table:
CREATE TABLE ZZ_TEST_EmployeeAllocation(
	employee_number NVARCHAR(255), 
	employee_name NVARCHAR(255), 
	Job NVARCHAR(255), 
	pay_group NVARCHAR(255), 
	location NVARCHAR(255), 
	location_code NVARCHAR(255), 
	annual NVARCHAR(255), 
	hourly NVARCHAR(255), 
	employment_status NVARCHAR(255), 
	allocation NVARCHAR(255), 
	job_code NVARCHAR(255), 
	location_code_2 NVARCHAR(255), 
	location2 NVARCHAR(255), 
	org_level_3_code NVARCHAR(255), 
	scheduled_work_hours NVARCHAR(255))